### Second Excersie of Langraph tutorial - LLM with Tools Binding

---

installing core dependencies

In [1]:
%%capture --no-stderr
%pip install -q langchain_google_genai langchain_core langgraph langchain_community

If you're using google colab so load environment in that way

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

if you're working locally so you need to load environment in that way

In [3]:
%%capture --no-stderr
%pip install python-dotenv

create `.env` file in project directory

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env file

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

I'm using Gemini Model, so you can use what you like, make sure to install langchain specific package for require Model, and set `API_KEY` accordingly

- Now Initializing Gemini LLM

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temprature=0.4)

I'm creating tool function which I'll be using with llm

In [13]:
# Please make sure to put DOCSTRING in the function, so that the function can be used by LLM
def deposit_money(acc_number:str, amount:int, acc_holder_name:str):
    """
    This is a tool function which should work with LLM
    This function will deposit money to the account

    Args:
        acc_number: str : Account number
        amount: int : Amount to deposit
        acc_holder_name: str : Account holder name

    Returns:
        str : Deposit message
    """
    # Your Actual Logic Here
    # ======================
    return f"Deposited {amount} to account {acc_number} of {acc_holder_name}"

Now I have to bind this tool with llm

In [14]:
llm_t = llm.bind_tools([deposit_money])

I need to test the LLM in a money deposit context. First, I'll evaluate it without using any tools, and then with the tool enabled. This will help highlight the differences in performance.

In [15]:
# Note: I'm invoking LLM without tool
llm.invoke("I want to deposit 1000 to account 123456 of John Doe")

AIMessage(content="Please note that I am a large language model, an AI.  I cannot directly access or interact with any real bank accounts. I cannot make deposits for you.\n\nTo deposit $1000 into account 123456 for John Doe, you will need to use one of the methods provided by the financial institution holding the account. This could include:\n\n* **Online Banking:** Log into your online banking account and use the transfer or deposit feature.\n* **Mobile Banking App:** Use the bank's mobile app to make a transfer or deposit.\n* **ATM Deposit:**  Deposit cash or checks at an ATM associated with the account's bank.\n* **In-Person Deposit:** Visit a branch of the bank and make a deposit with a teller.\n* **Mail a Check:** Mail a check payable to John Doe to the bank's address (be sure to include the account number).\n* **Wire Transfer:** Initiate a wire transfer through your bank to John Doe's account.\n\n\nYou'll need to ensure you have the necessary information and authorization to depo

In [16]:
# Note: I'm invoking LLM with tool
llm_t.invoke("I want to deposit 1000 to account 123456 of John Doe")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'deposit_money', 'arguments': '{"acc_holder_name": "John Doe", "amount": 1000.0, "acc_number": "123456"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a728ba5a-3150-48a4-8e1c-a880c6f27b6e-0', tool_calls=[{'name': 'deposit_money', 'args': {'acc_holder_name': 'John Doe', 'amount': 1000.0, 'acc_number': '123456'}, 'id': 'e1a4f897-73f6-4858-a6b6-97387eb291da', 'type': 'tool_call'}], usage_metadata={'input_tokens': 119, 'output_tokens': 20, 'total_tokens': 139, 'input_token_details': {'cache_read': 0}})

As you can see, when the LLM is invoked without the tool, it doesn't understand the 'money deposit' context, and the `AIMessage` response doesn't contain any additional arguments for any tool.

However, when the LLM is invoked with the tool enabled, it correctly understands the 'money deposit' context, and the `AIMessage` response includes additional arguments for the tool that was called.